# CARGA VENTAS A BRONZE

In [0]:
import re
from pyspark.sql.functions import col

ruta_volumen = "/Volumes/workspace/bronze/uploads/Ventas.csv"

# 1. Leer el CSV
df_raw = (spark.read.format("csv")
    .option("header", "true")
    .option("sep", ";")
    .option("encoding", "ISO-8859-1")
    .load(ruta_volumen))

# 2. LIMPIEZA DE NOMBRES DE COLUMNAS (Solución al error del punto)
# Iteramos sobre df_raw.columns directamente para renombrar antes de cualquier operación
for old_name in df_raw.columns:
    # Transformación: "Num. Factura" -> "num_factura"
    new_name = old_name.lower().replace('á','a').replace('é','e').replace('í','i').replace('ó','o').replace('ú','u')
    # Reemplaza puntos, espacios y caracteres no alfanuméricos por "_"
    new_name = re.sub(r'[^a-z0-9]+', '_', new_name).strip('_')
    df_raw = df_raw.withColumnRenamed(old_name, new_name)

# 3. Eliminar columnas fantasma (las que quedaron vacías por los ";;")
cols_a_mantener = [c for c in df_raw.columns if not c.startswith("_c")]
df_final_bronze = df_raw.select(*cols_a_mantener)

# 4. Guardar en Bronze
df_final_bronze.write.mode("overwrite").option("overwriteSchema", "true").saveAsTable("workspace.bronze.ventas")

print("✅ Tabla Ventas en Bronze creada")